In [7]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv
import re

In [132]:
# Set the target date 
target_date = datetime.now() + timedelta(days=5)
target_date = datetime.strftime(target_date, '%d.%m.%Y')

In [16]:
# Scraping the data from SRF. 
response = requests.get("http://m.srf.ch/meteo/lokalprognose?id=417255326")
soup_srf = BeautifulSoup(response.text, 'html.parser')

In [131]:
# Extract and format the parts for each hour
for i in range(2,24,3):
    temp_hour = soup_srf.find_all("li", attrs={'class':'hour-forecast','data-date':str(target_date),'data-hour':str(i)})

    # Grab the data for the description from img alt
    for item in temp_hour:
        temp_description = item.find('img', alt=True)
        temp_description = temp_description['alt']
    
    # Grab the temperature.
    for item in temp_hour:
        temp_temp = item.find("span", attrs={'class':'offscreen'})
        temp_temp = temp_temp.string.strip().replace('Durchschnittliche Temperatur ','').replace('°','')
    
    # Grab the rain by looking for the next sibling.
    for item in temp_hour:
        temp_rain = item.find_next_sibling('div')
        temp_rain = temp_rain.find('dd').text.replace(' %', '').split(' mm / ')

    # Grab the wind.(From second dd inside of 'div', attrs={'class':'forecast-data'})
    for item in temp_hour:
        temp_wind = item.find_next_sibling('div')
        temp_wind = temp_wind.find('dd').find_next_sibling('dd').text.replace(' km/h', '').split(' | ')

    # Generating the date.
    time = datetime.strptime(target_date, '%d.%m.%Y') + timedelta(hours=i)
    
    # Saving the data to a file.
    with open(r'srf.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([time,temp_temp,temp_description,float(temp_rain[0]),float(temp_rain[1]),float(temp_wind[0]),float(temp_wind[1])])